In [13]:
#NLP 파이프 라인 생성
import nltk
import threading
import queue
import feedparser
import uuid

In [8]:
threads = [] #단일프로그램에서 겨량 테스크를 생성하는데 사용
queues = [queue.Queue(), queue.Queue()] #드 개의 대기열 목록 생성

In [9]:
def extractWords():
    url = 'https://timesofindia.indiatimes.com/rssfeeds/1081479906.cms'
    feed = feedparser.parse(url)
    for entry in feed['entries'][:5]:
        text = entry['title']
        if 'ex' in text:
            continue
        words = nltk.word_tokenize(text)
        data = {'uuid': uuid.uuid4(), 'input': words}
        queues[0].put(data, True)
        print(">> {} : {}".format(data['uuid'], text))

In [10]:
def extractPOS():
    while True:
        if queues[0].empty():
            break
        else:
            #첫번째 대기열에서 데이터를 읽어온 후, 
            #두번째 대기열에서 단어의 품사 저장
            data = queues[0].get()
            words = data['input']
            postags = nltk.pos_tag(words)
            queues[0].task_done()
            queues[1].put({'uuid': data['uuid'], 'input': postags}, True)

In [11]:
def extractNE():
    while True:
        if queues[1].empty():
            break
        else:
            #두번째 대기열에서 품사 태그가 지정된 단어를 처리하여 개체명 출력
            data = queues[1].get()
            postags = data['input']
            queues[1].task_done()
            chunks = nltk.ne_chunk(postags, binary=False)
            print("  << {} : ".format(data['uuid']), end = '')
            for path in chunks:
                try:
                    label = path.label()
                    print(path, end=', ')
                except:
                    pass
            print()

In [12]:
#1.각각의 함수를 갖는 Thread를 생성
#2.각 Thread에 개체를 저장
#3.join()함수를 호출해 대기열에 할당된 자원을 해제
def runProgram():
    e = threading.Thread(target=extractWords())
    e.start()
    threads.append(e)

    p = threading.Thread(target=extractPOS())
    p.start()
    threads.append(p)

    n = threading.Thread(target=extractNE())
    n.start()
    threads.append(n)
    
    #5개의 교유 id,text 데이터와 chunking된 결과를 출력
    queues[0].join()
    queues[1].join()

    for t in threads:
        t.join()

In [7]:
if __name__ == '__main__':
    runProgram()

>> 8851e9a1-bdec-4fca-8416-150b14112203 : SRK & Arbaaz's dad share the same birthday
>> 27e8adce-b19a-43e8-a4d1-5093ac7490bd : Twitter shares memes on Anushka-Virat
>> 1400d661-2656-4990-8a8e-d3e53e9460c0 : Fans of Alia Bhatt discover her lookalike
>> b9ecd084-a7a5-4da1-996b-e1010b63fd5f : Newlyweds who will celebrate their 1st Diwali
>> 4739dbbd-afee-4e65-bcd2-36e13acb9256 : Hansal Mehta: SRK is my superstar forever
  << 8851e9a1-bdec-4fca-8416-150b14112203 : (ORGANIZATION SRK/NNP), (PERSON Arbaaz/NNP), 
  << 27e8adce-b19a-43e8-a4d1-5093ac7490bd : 
  << 1400d661-2656-4990-8a8e-d3e53e9460c0 : (PERSON Alia/NNP Bhatt/NNP), 
  << b9ecd084-a7a5-4da1-996b-e1010b63fd5f : 
  << 4739dbbd-afee-4e65-bcd2-36e13acb9256 : (PERSON Hansal/NNP), (PERSON Mehta/NNP), 
